In [86]:
import pandas as pd
import textblob as tb
import nltk

In [87]:
df = pd.read_csv('../../6-NLP/data/Hotel_Reviews.csv')
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [88]:
dropped_columns = ['lat','lng','Additional_Number_of_Scoring']
df.drop(dropped_columns, axis=1, inplace=True)

In [89]:
def replace_address(row):
    if "Netherlands" in row["Hotel_Address"]:
        return "Amsterdam, Netherlands"
    elif "Barcelona" in row["Hotel_Address"]:
        return "Barcelona, Spain"
    elif "United Kingdom" in row["Hotel_Address"]:
        return "London, United Kingdom"
    elif "Milan" in row["Hotel_Address"]:        
        return "Milan, Italy"
    elif "France" in row["Hotel_Address"]:
        return "Paris, France"
    elif "Vienna" in row["Hotel_Address"]:
        return "Vienna, Austria" 

In [90]:
df['Hotel_Address'] = df.apply(replace_address, axis=1)
print(df['Hotel_Address'].value_counts())

London, United Kingdom    262301
Barcelona, Spain           60149
Paris, France              59928
Amsterdam, Netherlands     57214
Vienna, Austria            38939
Milan, Italy               37207
Name: Hotel_Address, dtype: int64


In [91]:
display(df.groupby("Hotel_Address").agg({"Hotel_Name": "nunique"}))

,Hotel_Name
Hotel_Address,
"Amsterdam, Netherlands",105
"Barcelona, Spain",211
"London, United Kingdom",400
"Milan, Italy",162
"Paris, France",458
"Vienna, Austria",158


In [92]:
df.Total_Number_of_Reviews = df.groupby("Hotel_Name").transform('count')['Total_Number_of_Reviews']
df.Average_Score = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)

In [93]:
df.drop(['Review_Total_Negative_Word_Counts','Review_Total_Positive_Word_Counts','Review_Date','days_since_review'],axis=1,inplace=True)

In [94]:
df.drop(['Total_Number_of_Reviews_Reviewer_Has_Given'],axis=1,inplace=True)

In [95]:
# Remove opening and closing brackets
df.Tags = df.Tags.str.strip("[']")
# remove all quotes too
df.Tags = df.Tags.str.replace(" ', '", ",", regex = False)

In [96]:
tag_count = df.Tags.str.split(",", expand=True)

df['Tag_1'] = tag_count[0].str.strip()
df['Tag_2'] = tag_count[1].str.strip()
df['Tag_3'] = tag_count[2].str.strip()
df['Tag_4'] = tag_count[3].str.strip()
df['Tag_5'] = tag_count[4].str.strip()
df['Tag_6'] = tag_count[5].str.strip()

In [97]:
df_tags = df.melt(value_vars=["Tag_1", "Tag_2", "Tag_3", "Tag_4", "Tag_5", "Tag_6"])
# Drop rooms, suites, and length of stay, mobile device and anything with less count than a 1000
df_tags = df_tags[~df_tags.value.str.contains("Standard|room|Stayed|device|Beds|Suite|Studio|King|Superior|Double", na=False, case=False)]
tag_vc = df_tags.value.value_counts().reset_index(name="count").query("count > 1000")
# Print the top 10 (there should only be 9 and we'll use these in the filtering section)
print(tag_vc[:10])


                        index   count
0                Leisure trip  417778
1                      Couple  252294
2               Solo traveler  108545
3               Business trip   82939
4                       Group   65392
5  Family with young children   61015
6  Family with older children   26349
7      Travelers with friends    2143
8                  With a pet    1405


In [98]:
# Process the Tags into new columns
# The file Hotel_Reviews_Tags.py, identifies the most important tags
# Leisure trip, Couple, Solo traveler, Business trip, Group combined with Travelers with friends, 
# Family with young children, Family with older children, With a pet
df["Leisure_trip"] = df.Tags.apply(lambda tag: 1 if "Leisure trip" in tag else 0)
df["Couple"] = df.Tags.apply(lambda tag: 1 if "Couple" in tag else 0)
df["Solo_traveler"] = df.Tags.apply(lambda tag: 1 if "Solo traveler" in tag else 0)
df["Business_trip"] = df.Tags.apply(lambda tag: 1 if "Business trip" in tag else 0)
df["Group"] = df.Tags.apply(lambda tag: 1 if "Group" in tag or "Travelers with friends" in tag else 0)
df["Family_with_young_children"] = df.Tags.apply(lambda tag: 1 if "Family with young children" in tag else 0)
df["Family_with_older_children"] = df.Tags.apply(lambda tag: 1 if "Family with older children" in tag else 0)
df["With_a_pet"] = df.Tags.apply(lambda tag: 1 if "With a pet" in tag else 0)

In [99]:
df.drop(['Tag_1','Tag_2','Tag_3','Tag_4','Tag_5','Tag_6'], axis = 1, inplace=True)

# Saving new data file with calculated columns
print("Saving results to Hotel_Reviews_Filtered.csv")
df.to_csv(r'../data/Hotel_Reviews_Filtered.csv', index = False)

Saving results to Hotel_Reviews_Filtered.csv


In [101]:
df.drop(['Tags'], axis = 1, inplace=True)
df.head()

,Hotel_Address,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Total_Number_of_Reviews,Positive_Review,Reviewer_Score,Leisure_trip,Couple,Solo_traveler,Business_trip,Group,Family_with_young_children,Family_with_older_children,With_a_pet
0,"Amsterdam, Netherlands",7.8,Hotel Arena,Russia,I am so angry that i made this post available...,405,Only the park outside of the hotel was beauti...,2.9,1,1,0,0,0,0,0,0
1,"Amsterdam, Netherlands",7.8,Hotel Arena,Ireland,No Negative,405,No real complaints the hotel was great great ...,7.5,1,1,0,0,0,0,0,0
2,"Amsterdam, Netherlands",7.8,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,405,Location was good and staff were ok It is cut...,7.1,1,0,0,0,0,1,0,0
3,"Amsterdam, Netherlands",7.8,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,405,Great location in nice surroundings the bar a...,3.8,1,0,1,0,0,0,0,0
4,"Amsterdam, Netherlands",7.8,Hotel Arena,New Zealand,You When I booked with your company on line y...,405,Amazing location and building Romantic setting,6.7,1,1,0,0,0,0,0,0
